In [ ]:
import re
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup as bs
from vac_name_cleener import vac_name_cleener
from HH_vac_block_parcer import get_info

In [ ]:
vacancy_name = 'sql'

In [ ]:
vacancy_name = vac_name_cleener(vacancy_name)

vacancy_list = []
salary_list = []
min_salary = []
max_salary = []
url_list = []
currency_list = []
source_name_list = []
employer_list = []


In [ ]:
pages = 1  # !!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
domain = 'hh.ru'
main_link = 'https://hh.ru/search/vacancy?text='

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) \
     AppleWebKit/537.36 (KHTML, like Gecko) \
     Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
}

In [ ]:
pg_num = '2'

In [ ]:
page = str(pg_num);
page

In [ ]:
link = main_link + vacancy_name + '&page=' + page
link

In [ ]:
html = requests.get(link, headers=headers)
html

In [ ]:
soup = bs(html.text, 'lxml')

In [ ]:
htmlout = str(soup)
with open("soup.html", "w") as file:
    file.write(htmlout)

In [ ]:
vacancy_block = soup.find_all('div', class_='vacancy-serp-item') # 50 vacs was found

In [ ]:
len(vacancy_block)

### * * * * * * 

In [ ]:
vacancy = vacancy_block[8]

In [ ]:
len(vacancy)

In [ ]:
# должность
vacancy.find_all('a', href=True)[0].text

In [ ]:
for i in range(len(vacancy.find_all('a', href=True))):
    print(i, vacancy.find_all('a', href=True)[i].text)

In [ ]:
# работодатель
employers_tags = {"class":"bloko-link bloko-link_secondary", "data-qa":"vacancy-serp__vacancy-employer"}
vacancy.find("a", employers_tags).text

In [ ]:
<span class="vacancy-serp-item__meta-info" data-qa="vacancy-serp__vacancy-address">Саратов</span>

In [ ]:
salary_tags = {'class':'bloko-section-header-3 bloko-section-header-3_lite',
               'data-qa':'vacancy-serp__vacancy-compensation'}
vacancy.find('span', salary_tags) is None

In [ ]:
vacancy.find(class_="bloko-link bloko-link_secondary").text

In [ ]:
vacancy.find('span', salary_tags).text

In [ ]:
salaryblock = vacancy.find('span', salary_tags).text


if salaryblock[:2] == 'от':
    salary_min = ''.join(re.findall(r'[0-9]', salaryblock))
    salary_max = None
elif salaryblock[:2] == 'до':
    salary_min = None
    salary_max = ''.join(re.findall(r'[0-9]', salaryblock))
else:
    salary = salaryblock.split('-')
    if len(salary) == 1:
        salary_min = salary[0]
        salary_max = salary[0]
    else:
        salary_min = ''.join(re.findall(r'[0-9]', salary[0]))
        salary_max = ''.join(re.findall(r'[0-9]', salary[-1]))
salary_min = int(salary_min)
salary_max = int(salary_max)
currency = salaryblock[-4:]      

In [ ]:
salaryblock

In [ ]:
print(salary_min)
print(salary_max)
print(currency)

In [ ]:
type(salary_min)

In [ ]:
link = vacancy.find_all('a', href=True)[0].get('href')

In [ ]:
link

In [ ]:
vacancy.find(class_="bloko-link bloko-link_secondary").text

In [ ]:
vacancy.find({'class':'bloko-link bloko-link_secondary'}).text

# superjob

In [2]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import time

from getpage import getpage
from name_cleener import sj_name_cleener

### Временное
vacancy_name = "c#"
pg_num = 0
###

# создаем запрос и получаем ответ
domain = 'https://www.superjob.ru'
main_link = 'https://www.superjob.ru/vacancy/search/?keywords='
page = str(pg_num)
link = main_link + sj_name_cleener(vacancy_name) + '&page=' + str(page)
print(f'Conneting to page № {pg_num}. {link}')
bsObj = getpage(link)

# создаем список элементов с названиями вакансий и  URL
vacancy_list=[]
url_list=[]

tag = 'div'
class_attrs = {'class':'_3mfro PlM3e _2JVkc _3LJqf'}
vacancy_block = bsObj.findAll(tag, class_attrs)

for vacancy in vacancy_block:
    vacancy_list.append(vacancy.getText())
    url_list.append(domain + vacancy.find('a', href=True)['href'])

# создаем список работодателей
employer_list=[]
employer_block = bsObj.findAll(class_=employer_re)

for i,employer in enumerate(employer_block):
    employer_list.append(employer_block[i].getText())

# создаем список ссылок на источник данных
source_name_list = [domain] * 20

# создаем список элементов с информациям по зарплатам
salary_list = []
tag = 'span'
class_attrs = {'class':'_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'}
salary_block = bsObj.findAll(tag, class_attrs)

for i,salary in enumerate(salary_block):
    salary_list.append(salary.getText())
    salary_list[i] = re.sub("\xa0","",salary_list[i])

min_salary = []
max_salary = []
currency_list = []

for salary in salary_list:
    if salary == 'По договорённости':
        salary_min = None
        salary_max = None
        currency = None

    else:
        if salary[:2] == 'от':
            salary_min = int(''.join(re.findall(r'[0-9]', salary)))
            salary_max = None
            currency = re.findall('[\D]+$', salary)[-1]

        elif salary[:2] == 'до':
            salary_min = None
            salary_max = int(''.join(re.findall(r'[0-9]', salary)))
            currency = re.findall('[\D]+$', salary)[-1]

        else:
            currency = re.findall('[\D]+$', salary)[-1]
            salary = re.split(r'—',salary)

            if len(salary) == 1:
                salary_min = int(''.join(re.findall(r'[0-9]', salary[0])))
                salary_max = int(''.join(re.findall(r'[0-9]', salary[0])))

            else:
                salary_min = int(''.join(re.findall(r'[0-9]', salary[0])))
                salary_max = int(''.join(re.findall(r'[0-9]', salary[-1])))
      
    min_salary.append(salary_min)
    max_salary.append(salary_max)
    currency_list.append(currency)

# Создаем результирующий датафрейм
df = pd.DataFrame()
df['vacancy'] = pd.Series(vacancy_list)
df['min_salary'] = pd.Series(min_salary)
df['max_salary'] = pd.Series(max_salary)
df['currency'] = pd.Series(currency_list)
df['url'] = pd.Series(url_list)
df['source'] = pd.Series(source_name_list)
df['employer'] = pd.Series(employer_list)

# Делаем пейджератор SJ

In [ ]:
<span class="_3IDf-">Дальше</span>

In [ ]:
<span class="_23m0W"><span class="_3IDf-">Дальше</span></span>

In [309]:
tag = 'span'
#class_attrs = {'class':'_3IDf-'}
class_attrs = {'class':'_23m0W'}

In [319]:
bsObj.findAll(tag, class_attrs)[-4]

<span class="_23m0W"><span class="_3IDf-">Дальше</span></span>

In [316]:
bsObj.find('_23m0W')

In [ ]:
#app > div > div._3R0rZ > div._1X8YL > div > div:nth-child(1) > div.iJCa5.undefined._1LlO2.X7voU._2nteL > div._3Qutk > div._1Ttd8._2CsQi > div._3zucV.undefined._3_4D0._3SGgo > div > div > a.icMQ_._1_Cht._3ze9n.f-test-button-dalshe.f-test-link-Dalshe > span > span

In [330]:
a = bsObj.find(text='Дальше').parent.parent.parent.parent
a['href']

'/vacancy/search/?keywords=c%23&page=2'

In [332]:
a = bsObj.find(text='Дальше').parent.parent.parent.parent['href']
a

'/vacancy/search/?keywords=c%23&page=2'

In [333]:
domain+a

'https://www.superjob.ru/vacancy/search/?keywords=c%23&page=2'

In [ ]:
def sj_parcer(vacancy_name):
    # создаем запрос и получаем ответ
    domain = 'https://www.superjob.ru'
    link_suffix = '/vacancy/search/?keywords='
    link = domain + link_suffix + sj_name_cleener(vacancy_name)
    
    df, link = sj_page_parcer(link)
    while link

In [ ]:
def sj_page_parcer(link):
    
    

In [338]:

link_suffix = '/vacancy/search/?keywords='
link = domain + link_suffix + sj_name_cleener(vacancy_name) 
link

'https://www.superjob.ru/vacancy/search/?keywords=c%23'

In [343]:
while link:
    link = link[:-1]
    print(link)
    print('ok')

https://www.superjob.ru/vacancy/search/?keywords=c%2
ok
https://www.superjob.ru/vacancy/search/?keywords=c%
ok
https://www.superjob.ru/vacancy/search/?keywords=c
ok
https://www.superjob.ru/vacancy/search/?keywords=
ok
https://www.superjob.ru/vacancy/search/?keywords
ok
https://www.superjob.ru/vacancy/search/?keyword
ok
https://www.superjob.ru/vacancy/search/?keywor
ok
https://www.superjob.ru/vacancy/search/?keywo
ok
https://www.superjob.ru/vacancy/search/?keyw
ok
https://www.superjob.ru/vacancy/search/?key
ok
https://www.superjob.ru/vacancy/search/?ke
ok
https://www.superjob.ru/vacancy/search/?k
ok
https://www.superjob.ru/vacancy/search/?
ok
https://www.superjob.ru/vacancy/search/
ok
https://www.superjob.ru/vacancy/search
ok
https://www.superjob.ru/vacancy/searc
ok
https://www.superjob.ru/vacancy/sear
ok
https://www.superjob.ru/vacancy/sea
ok
https://www.superjob.ru/vacancy/se
ok
https://www.superjob.ru/vacancy/s
ok
https://www.superjob.ru/vacancy/
ok
https://www.superjob.ru/vacancy
ok

In [342]:
link[:-1]

'https://www.superjob.ru/vacancy/search/?keywords=c'

In [352]:
res = pd.DataFrame()
res

""


In [356]:
res = pd.concat([res, df],ignore_index=True)
res

,vacancy,min_salary,max_salary,currency,url
0,Программист C# (для разработки систем АИИС КУЭ...,NaN,150000.0,руб.,https://www.superjob.ru/vakansii/programmist-c...
1,Программист C#,100000.0,NaN,руб.,https://www.superjob.ru/vakansii/programmist-c...
2,Senior C# /.NET developer,NaN,NaN,None,https://www.superjob.ru/vakansii/senior-c-3458...
3,"Ведущий разработчик C#, .NET",140000.0,160000.0,руб.,https://www.superjob.ru/vakansii/veduschij-raz...
4,Программист C#,NaN,120000.0,руб.,https://www.superjob.ru/vakansii/programmist-c...
...,...,...,...,...,...
75,Программист 1C,100000.0,100000.0,руб.,https://www.superjob.ru/vakansii/programmist-1...
76,Руководитель группы разработки ПО (Team lead),NaN,NaN,None,https://www.superjob.ru/vakansii/rukovoditel-g...
77,Инженер-программист С/С++ /Программист микроко...,NaN,150000.0,руб.,https://www.superjob.ru/vakansii/inzhener-prog...
78,"Инженер, старший инженер (отдел аппаратно-прог...",45000.0,55000.0,руб.,https://www.superjob.ru/vakansii/inzhener-3008...


In [370]:
import re
import pandas as pd
import time

from getpage import getpage
from name_cleener import sj_name_cleener

domain = 'https://www.superjob.ru'


def sj_page_parcer(link):
    # print(f'Conneting to page {link}')
    bsObj = getpage(link)

    # создаем список элементов с названиями вакансий и  URL
    vacancy_list = []
    url_list = []

    tag = 'div'
    class_attrs = {'class': '_3mfro PlM3e _2JVkc _3LJqf'}
    vacancy_block = bsObj.findAll(tag, class_attrs)

    for vacancy in vacancy_block:
        vacancy_list.append(vacancy.getText())
        url_list.append(domain + vacancy.find('a', href=True)['href'])

    # создаем список работодателей
    employer_list = []
    employer_block = bsObj.findAll(class_=employer_re)

    for i, employer in enumerate(employer_block):
        employer_list.append(employer_block[i].getText())

    # создаем список ссылок на источник данных
    source_name_list = [domain] * 20

    # создаем список элементов с информациям по зарплатам
    salary_list = []
    tag = 'span'
    class_attrs = {'class': '_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'}
    salary_block = bsObj.findAll(tag, class_attrs)

    for i, salary in enumerate(salary_block):
        salary_list.append(salary.getText())
        salary_list[i] = re.sub("\xa0", "", salary_list[i])

    min_salary = []
    max_salary = []
    currency_list = []

    for salary in salary_list:
        if salary == 'По договорённости':
            salary_min = None
            salary_max = None
            currency = None

        else:
            if salary[:2] == 'от':
                salary_min = int(''.join(re.findall(r'[0-9]', salary)))
                salary_max = None
                currency = re.findall('[\D]+$', salary)[-1]

            elif salary[:2] == 'до':
                salary_min = None
                salary_max = int(''.join(re.findall(r'[0-9]', salary)))
                currency = re.findall('[\D]+$', salary)[-1]

            else:
                currency = re.findall('[\D]+$', salary)[-1]
                salary = re.split(r'—', salary)

                if len(salary) == 1:
                    salary_min = int(''.join(re.findall(r'[0-9]', salary[0])))
                    salary_max = int(''.join(re.findall(r'[0-9]', salary[0])))

                else:
                    salary_min = int(''.join(re.findall(r'[0-9]', salary[0])))
                    salary_max = int(''.join(re.findall(r'[0-9]', salary[-1])))

        min_salary.append(salary_min)
        max_salary.append(salary_max)
        currency_list.append(currency)

    # Создаем результирующий датaфрейм
    df = pd.DataFrame()
    df['vacancy'] = pd.Series(vacancy_list)
    df['min_salary'] = pd.Series(min_salary)
    df['max_salary'] = pd.Series(max_salary)
    df['currency'] = pd.Series(currency_list)
    df['url'] = pd.Series(url_list)
    df['source'] = pd.Series(source_name_list)
    df['employer'] = pd.Series(employer_list)

    # обрабатываем пейджератор
    try:
        next_link = domain + bsObj.find(text='Дальше').parent.parent.parent.parent['href']
    except:
        next_link = None
    time.sleep(1)
    return df, next_link


def sj_parcer(vacancy_name):
    # Запускаем проход по страницам:
    df = pd.DataFrame()
    link_suffix = '/vacancy/search/?keywords='
    link = domain + link_suffix + sj_name_cleener(vacancy_name)
    res, link = sj_page_parcer(link)
    df = pd.concat([res, df], ignore_index=True)
    while link:
        res, link = sj_page_parcer(link)
        df = pd.concat([res, df], ignore_index=True)
    return df

### ОТЛАДКА
df = sj_parcer('sql')
print(df)


                                              vacancy  min_salary  max_salary  \
0                         Ведущий инженер-программист     55000.0         NaN   
1                        Программист Visual Basic.Net    100000.0         NaN   
2           Программист (клиент-серверная разработка)         NaN         NaN   
3                              Разработчик отчетности         NaN         NaN   
4                                     Программист C++         NaN         NaN   
5                             Системный администратор     80000.0         NaN   
6       Специалист первой линии технической поддержки     24800.0     24800.0   
7         Специалист отдела информационных технологий     43000.0     45000.0   
8                                      Программист 1C    100000.0    100000.0   
9                             Middle Delphi Developer    100000.0    120000.0   
10                                Инженер-программист         NaN         NaN   
11      Руководитель группы 

# *** pagerator for HH

In [373]:
import pandas as pd
import time

from name_cleener import hh_name_cleener
from HH_vac_block_parcer import get_info
from getpage import getpage

domain = 'https://www.hh.ru'


def hh_page_parcer(link):

    # Создаем списки целевых показателей:
    vacancy_list = []
    min_salary = []
    max_salary = []
    url_list = []
    currency_list = []
    source_name_list = []
    employer_list = []

    print(f'Conneting to page {link}')
    soup = getpage(link)

    # ищем блок с вакансиями
    vacancy_block = soup.find_all('div', class_='vacancy-serp-item')

    for vacancy in vacancy_block:
        vac_name, salary_min, salary_max, currency, link, employer = get_info(vacancy)
        vacancy_list.append(vac_name)
        min_salary.append(salary_min)
        max_salary.append(salary_max)
        currency_list.append(currency)
        url_list.append(link)
        source_name_list.append(domain)
        employer_list.append(employer)

    # Создаем результирующий датафрейм
    df = pd.DataFrame()
    df['vacancy'] = pd.Series(vacancy_list)
    df['min_salary'] = pd.Series(min_salary)
    df['max_salary'] = pd.Series(max_salary)
    df['currency'] = pd.Series(currency_list)
    df['url'] = pd.Series(url_list)
    df['source'] = pd.Series(source_name_list)
    df['employer'] = pd.Series(employer_list)
    
    # обрабатываем пейджератор
    try:
        next_link = domain + bsObj.find(text='Дальше').parent.parent.parent.parent['href']
    except:
        next_link = None

    time.sleep(1)
    return df, next_link

def hh_parcer(vacancy_name):
    # Запускаем проход по страницам:
    df = pd.DataFrame()
    link_suffix = main_link = '/search/vacancy?text='
    link = domain + link_suffix + hh_name_cleener(vacancy_name)
    res, link = hh_page_parcer(link)
    df = pd.concat([res, df], ignore_index=True)
    while link:
        res, link = sj_page_parcer(link)
        df = pd.concat([res, df], ignore_index=True)
    print(f'{len(df)} vacancies was found')
    return df

In [374]:
hh_parcer('sql')

Conneting to page https://www.hh.ru/search/vacancy?text=sql
('SQL Server DBA Bi Developer', None, None, None, 'https://spb.hh.ru/vacancy/39706709?query=sql', ' ARGO - торговая сеть') is parced
('Разработчик Oracle (PL/SQL)', 2000, 3000, ' USD', 'https://spb.hh.ru/vacancy/39468435?query=sql', ' Кадровое агентство Алексея Сухорукова') is parced
('Разработчик PL/SQL', 120000, 150000, 'руб.', 'https://spb.hh.ru/vacancy/39650419?query=sql', ' Открытые Технологии') is parced
('Разработчик SQL', None, None, None, 'https://spb.hh.ru/vacancy/39940525?query=sql', 'ООО АБП') is parced
('Программист SQL', None, None, None, 'https://spb.hh.ru/vacancy/40008083?query=sql', 'ООО Аврора-ИТ') is parced
('Разработчик Oracle PL/SQL', None, None, None, 'https://spb.hh.ru/vacancy/39956055?query=sql', ' Hyundai GLOVIS') is parced


AttributeError: 'NoneType' object has no attribute 'text'